In [1]:
from scipy.optimize import minimize
import joblib
import numpy as np
%run 2.Hicks_Henne_NEW_Final.ipynb
%run 3.Xfoil_runner_extract_value.ipynb

In [2]:
def get_new_sobol_samples(limit, num_samples, seed=None):
    l00, l01 = limit[0][0], limit[0][1]
    l10, l11 = limit[1][0], limit[1][1]
    l20, l21 = limit[2][0], limit[2][1]
    l30, l31 = limit[3][0], limit[3][1]
    l40, l41 = limit[4][0], limit[4][1]
    l50, l51 = limit[5][0], limit[5][1] 
    l60, l61 = limit[6][0], limit[6][1]
    l70, l71 = limit[7][0], limit[7][1]
    
    bounds = {"x1": [l00, l01], "x2": [l10, l11], "x3": [l20, l21], "x4": [l30, l31], 
              "x5": [l40, l41], "x6": [l50, l51], "x7": [l60, l61], "x8": [l70, l71]}
    sampler = Sobol(len(bounds), seed=seed)
    samples = sampler.random(num_samples)
    
    l_bounds = [bound[0] for bound in bounds.values()]
    u_bounds = [bound[1] for bound in bounds.values()]
    samples = qmc.scale(samples, l_bounds, u_bounds)
    design_parameter = pd.DataFrame(samples, columns=list(bounds.keys())).round(4)
    a = design_parameter.apply(np.array, axis=1)
    return a


In [8]:
new_dataset = pd.read_excel('new_dataset.xlsx')
new_dataset.drop(columns=['Unnamed: 0'], inplace=True)
new_dataset

,x1,x2,x3,x4,x5,x6,x7,x8,CL,CD
0,0.008400,0.010400,-0.003300,-0.003300,-0.008600,0.001500,-0.001900,-0.002700,0.5294,0.00733
1,-0.004500,-0.003500,0.005700,0.006200,0.010600,-0.006000,0.010100,0.003700,0.6747,0.00947
2,-0.009300,0.003800,-0.008300,-0.011300,0.003000,-0.002800,-0.009300,0.008500,0.4693,0.00959
3,0.001900,-0.009300,0.006700,0.002500,-0.001200,0.007700,0.002600,-0.009200,0.5034,0.00928
4,0.004500,0.001700,0.002600,0.005000,-0.003900,0.003400,-0.008700,0.010500,0.6249,0.00916
...,...,...,...,...,...,...,...,...,...,...
121,-0.004200,0.009000,-0.004000,-0.001300,0.001600,-0.000200,0.007100,0.005700,0.6374,0.00880
122,0.008600,-0.001900,0.004900,0.010300,-0.005800,0.007200,-0.004800,-0.000700,0.5924,0.00868
123,-0.012000,-0.012000,0.007357,0.012000,-0.012000,-0.007985,0.006126,0.006660,0.5498,0.01109
124,-0.011454,-0.011999,0.006688,0.012000,-0.009981,-0.008300,0.003945,0.007178,0.5475,0.01094


In [19]:
length = len(new_dataset)
length

126

In [20]:
X = new_dataset[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'] ]
X = X.apply(np.array, axis = 1)
new_sample_param = X[[length-3,length-2,length-1]]
new_sample_param

123    [-0.01199999999999997, -0.01199999994779628, 0...
124    [-0.01145381219301164, -0.01199935566071041, 0...
125    [-0.01001387667216669, -0.009139258396986339, ...
dtype: object

In [21]:
disp = 0.001
new_num_samples = 4
for i in [length-3,length-2,length-1]:
    limit= []
    a_new = X[i]
    for j in a_new:
        l= []
        l.append(j-disp)
        l.append(j+disp)
        limit.append(l)
    s_new = get_new_sobol_samples(limit, new_num_samples, seed=42)
    new_sample_param = np.append(new_sample_param,s_new)
new_sample_param

array([array([-0.012     , -0.012     ,  0.00735729,  0.012     , -0.012     ,
              -0.00798543,  0.00612598,  0.00666   ])                         ,
       array([-0.01145381, -0.01199936,  0.00668844,  0.012     , -0.00998115,
              -0.00830045,  0.00394512,  0.00717842])                         ,
       array([-0.01001388, -0.00913926,  0.0088487 ,  0.01189317,  0.01033303,
              -0.00501175, -0.012     ,  0.00355748])                         ,
       array([-0.0121, -0.0114,  0.008 ,  0.0111, -0.0124, -0.0073,  0.0065,
               0.0069])                                                     ,
       array([-0.0111, -0.0127,  0.0073,  0.013 , -0.0111, -0.0081,  0.0052,
               0.0064])                                                     ,
       array([-0.0118, -0.0119,  0.0079,  0.0119, -0.0118, -0.0087,  0.007 ,
               0.0061])                                                     ,
       array([-0.013 , -0.012 ,  0.0064,  0.0122, -0.0127,

In [22]:
len(new_sample_param)

15

In [23]:
def deformed_foil_generator(x, y, n, a, w, xM):
    for i in range(len(a)):
        y_mod = hicks_henne_overall(x, y, n, a, i, w, xM)
        df = pd.DataFrame({'x': x, 'y_modified': y_mod})
        path = "Deformed_foils\\Deformed_airfoil_" + str(i+128) + ".dat"
        
        df.to_csv(path, index=False, sep=' ')

In [24]:
if globals().get('__file__', '') == '':
    deformed_foil_generator(x, y, n, new_sample_param, w, xM)

In [25]:
new_sample_param = np.vstack(new_sample_param)
new_sample_param

array([[-0.012     , -0.012     ,  0.00735729,  0.012     , -0.012     ,
        -0.00798543,  0.00612598,  0.00666   ],
       [-0.01145381, -0.01199936,  0.00668844,  0.012     , -0.00998115,
        -0.00830045,  0.00394512,  0.00717842],
       [-0.01001388, -0.00913926,  0.0088487 ,  0.01189317,  0.01033303,
        -0.00501175, -0.012     ,  0.00355748],
       [-0.0121    , -0.0114    ,  0.008     ,  0.0111    , -0.0124    ,
        -0.0073    ,  0.0065    ,  0.0069    ],
       [-0.0111    , -0.0127    ,  0.0073    ,  0.013     , -0.0111    ,
        -0.0081    ,  0.0052    ,  0.0064    ],
       [-0.0118    , -0.0119    ,  0.0079    ,  0.0119    , -0.0118    ,
        -0.0087    ,  0.007     ,  0.0061    ],
       [-0.013     , -0.012     ,  0.0064    ,  0.0122    , -0.0127    ,
        -0.0075    ,  0.0057    ,  0.0076    ],
       [-0.0116    , -0.0114    ,  0.0073    ,  0.0111    , -0.0104    ,
        -0.0076    ,  0.0043    ,  0.0074    ],
       [-0.0106    , -0.0127    

In [26]:
new_sample_param.shape

(15, 8)

In [27]:
len(new_sample_param)

15

In [28]:
columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']
new_sample_param = pd.DataFrame(new_sample_param, columns=columns)
new_sample_param

,x1,x2,x3,x4,x5,x6,x7,x8
0,-0.012000,-0.012000,0.007357,0.012000,-0.012000,-0.007985,0.006126,0.006660
1,-0.011454,-0.011999,0.006688,0.012000,-0.009981,-0.008300,0.003945,0.007178
2,-0.010014,-0.009139,0.008849,0.011893,0.010333,-0.005012,-0.012000,0.003557
3,-0.012100,-0.011400,0.008000,0.011100,-0.012400,-0.007300,0.006500,0.006900
4,-0.011100,-0.012700,0.007300,0.013000,-0.011100,-0.008100,0.005200,0.006400
5,-0.011800,-0.011900,0.007900,0.011900,-0.011800,-0.008700,0.007000,0.006100
6,-0.013000,-0.012000,0.006400,0.012200,-0.012700,-0.007500,0.005700,0.007600
7,-0.011600,-0.011400,0.007300,0.011100,-0.010400,-0.007600,0.004300,0.007400
8,-0.010600,-0.012700,0.006600,0.013000,-0.009100,-0.008400,0.003000,0.006900
9,-0.011200,-0.011900,0.007200,0.011900,-0.009800,-0.009100,0.004800,0.006600


In [29]:
new_sample_param.to_excel('new_sample_param.xlsx')

In [30]:
if globals().get('__file__', '') == '':
    # Save the current directory
    original_directory = os.getcwd()
    
    # Navigate to another folder
    os.chdir(original_directory + '\\Deformed_foils')

In [31]:
def new_deformed_foil_data_from_Xfoil(alpha, Re, n_iter, num_samples, file_name):
    #outputs
    CL = []
    CD = []
    for i in range(num_samples):
        airfoil_name = "Deformed_airfoil_"+str(i+128)
        cl_i, cd_i = Xfoil_runner(alpha, Re, n_iter, airfoil_name)
        CL.append(cl_i)
        CD.append(cd_i)
        print(i,"-------->",cl_i,cd_i)
    data = {'CL': CL,'CD': CD}
    CL_CD = pd.DataFrame(data)
    
    # Return to the original directory
    os.chdir(original_directory)
    CL_CD.to_excel(file_name)
    return CL_CD

In [32]:
if globals().get('__file__', '') == '':
    file_name = 'new_Cl_Cd.xlsx'
    num_samples = 15
    CL_CD = new_deformed_foil_data_from_Xfoil(alpha, Re, n_iter, num_samples, file_name)

0 --------> 0.5498 0.01109
1 --------> 0.5475 0.01094
2 --------> 0.5304 0.01028
3 --------> nan nan
4 --------> 0.5492 0.01098
5 --------> 0.5504 0.01109
6 --------> 0.5483 0.01125
7 --------> 0.5517 0.01095
8 --------> 0.5464 0.01088
9 --------> 0.5478 0.01093
10 --------> 0.5456 0.01112
11 --------> 0.5355 0.01029
12 --------> 0.5286 0.01027
13 --------> 0.5305 0.01025
14 --------> 0.5294 0.0105


In [33]:
new_df = pd.merge(new_sample_param, CL_CD, left_index=True, right_index=True)
new_df = new_df.dropna()
new_df

,x1,x2,x3,x4,x5,x6,x7,x8,CL,CD
0,-0.012000,-0.012000,0.007357,0.012000,-0.012000,-0.007985,0.006126,0.006660,0.5498,0.01109
1,-0.011454,-0.011999,0.006688,0.012000,-0.009981,-0.008300,0.003945,0.007178,0.5475,0.01094
2,-0.010014,-0.009139,0.008849,0.011893,0.010333,-0.005012,-0.012000,0.003557,0.5304,0.01028
4,-0.011100,-0.012700,0.007300,0.013000,-0.011100,-0.008100,0.005200,0.006400,0.5492,0.01098
5,-0.011800,-0.011900,0.007900,0.011900,-0.011800,-0.008700,0.007000,0.006100,0.5504,0.01109
6,-0.013000,-0.012000,0.006400,0.012200,-0.012700,-0.007500,0.005700,0.007600,0.5483,0.01125
7,-0.011600,-0.011400,0.007300,0.011100,-0.010400,-0.007600,0.004300,0.007400,0.5517,0.01095
8,-0.010600,-0.012700,0.006600,0.013000,-0.009100,-0.008400,0.003000,0.006900,0.5464,0.01088
9,-0.011200,-0.011900,0.007200,0.011900,-0.009800,-0.009100,0.004800,0.006600,0.5478,0.01093
10,-0.012400,-0.012000,0.005700,0.012200,-0.010700,-0.007900,0.003500,0.008100,0.5456,0.01112


In [34]:
new_dataset = pd.read_excel('new_dataset.xlsx')
new_dataset.drop(columns=['Unnamed: 0'], inplace=True)
new_dataset

,x1,x2,x3,x4,x5,x6,x7,x8,CL,CD
0,0.008400,0.010400,-0.003300,-0.003300,-0.008600,0.001500,-0.001900,-0.002700,0.5294,0.00733
1,-0.004500,-0.003500,0.005700,0.006200,0.010600,-0.006000,0.010100,0.003700,0.6747,0.00947
2,-0.009300,0.003800,-0.008300,-0.011300,0.003000,-0.002800,-0.009300,0.008500,0.4693,0.00959
3,0.001900,-0.009300,0.006700,0.002500,-0.001200,0.007700,0.002600,-0.009200,0.5034,0.00928
4,0.004500,0.001700,0.002600,0.005000,-0.003900,0.003400,-0.008700,0.010500,0.6249,0.00916
...,...,...,...,...,...,...,...,...,...,...
121,-0.004200,0.009000,-0.004000,-0.001300,0.001600,-0.000200,0.007100,0.005700,0.6374,0.00880
122,0.008600,-0.001900,0.004900,0.010300,-0.005800,0.007200,-0.004800,-0.000700,0.5924,0.00868
123,-0.012000,-0.012000,0.007357,0.012000,-0.012000,-0.007985,0.006126,0.006660,0.5498,0.01109
124,-0.011454,-0.011999,0.006688,0.012000,-0.009981,-0.008300,0.003945,0.007178,0.5475,0.01094


In [35]:
df_concatenated_dataset = pd.concat([new_dataset, new_df])
df_concatenated_dataset

,x1,x2,x3,x4,x5,x6,x7,x8,CL,CD
0,0.0084,0.0104,-0.0033,-0.0033,-0.0086,0.0015,-0.0019,-0.0027,0.5294,0.00733
1,-0.0045,-0.0035,0.0057,0.0062,0.0106,-0.0060,0.0101,0.0037,0.6747,0.00947
2,-0.0093,0.0038,-0.0083,-0.0113,0.0030,-0.0028,-0.0093,0.0085,0.4693,0.00959
3,0.0019,-0.0093,0.0067,0.0025,-0.0012,0.0077,0.0026,-0.0092,0.5034,0.00928
4,0.0045,0.0017,0.0026,0.0050,-0.0039,0.0034,-0.0087,0.0105,0.6249,0.00916
...,...,...,...,...,...,...,...,...,...,...
10,-0.0124,-0.0120,0.0057,0.0122,-0.0107,-0.0079,0.0035,0.0081,0.5456,0.01112
11,-0.0102,-0.0085,0.0095,0.0110,0.0099,-0.0043,-0.0116,0.0038,0.5355,0.01029
12,-0.0091,-0.0099,0.0088,0.0129,0.0112,-0.0051,-0.0130,0.0033,0.5286,0.01027
13,-0.0098,-0.0090,0.0093,0.0118,0.0106,-0.0058,-0.0111,0.0030,0.5305,0.01025


In [36]:
# df_concatenated_dataset.to_excel('new_dataset.xlsx')